## I will be using TF - term Frequency to transform the documents into tf-idf vectors, then compute the cosine similarity between them. Descriptions are short, precise and well defined they often contain the important keywords that are self descriptory.


Let's go ahead and use rstrip() plus a list comprehension to get a list of all the lines of  messages:

In [1]:
messages = [line.rstrip() for line in open('company_descriptions.txt')]
print(len(messages))

5


Print Messages

In [2]:
for message_no, message in enumerate(messages[:10]):
    print(message_no, message)
    print('\n')

(0, 'AAPL: Apple Inc. is an American multinational technology company headquartered in Cupertino, California, that designs, develops, and sells consumer electronics, computer software, and online services.')


(1, "MSFT: Microsoft Corporation (commonly referred to as Microsoft) is an American multinational technology company headquartered in Redmond, Washington, that develops, manufactures, licenses, supports and sells computer software, consumer electronics and personal computers and services.  Its best known software products are the Microsoft Windows line of operating systems, Microsoft Office office suite, and Internet Explorer and Edge web browsers.  Its flagship hardware products are the Xbox game consoles and the Microsoft Surface tablet lineup.  It is the world's largest software maker by revenue, and one of the world's most valuable companies.")


(2, 'BAC: Bank of America is an American multinational banking and financial services corporation headquartered in Charlotte, North

In [3]:
import pandas as pd

In [4]:
messages = pd.read_table('company_descriptions.txt',sep =':'
                           ,names=["Company", "Desc"])
messages.head()

,Company,Desc
0,AAPL,Apple Inc. is an American multinational techn...
1,MSFT,Microsoft Corporation (commonly referred to a...
2,BAC,Bank of America is an American multinational ...
3,YHOO,Yahoo Inc. is an American multinational techn...
4,GOOG,Google Inc. is an American multinational tech...


In [5]:
messages.describe()

,Company,Desc
count,5,5
unique,5,5
top,MSFT,Bank of America is an American multinational ...
freq,1,1


In [6]:
messages['length'] = messages['Desc'].apply(len)
messages.head()

,Company,Desc,length
0,AAPL,Apple Inc. is an American multinational techn...,195
1,MSFT,Microsoft Corporation (commonly referred to a...,644
2,BAC,Bank of America is an American multinational ...,792
3,YHOO,Yahoo Inc. is an American multinational techn...,235
4,GOOG,Google Inc. is an American multinational tech...,335


In [7]:
def text_process(mess):

    # Check characters to see if they are in punctuation
    nopunc = [char for char in mess if char not in string.punctuation]

    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    
    # Now just remove any stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [8]:
import string
from nltk.corpus import stopwords
# Check to make sure its working
messages['Desc'].head(5).apply(text_process)

0    [Apple, Inc, American, multinational, technolo...
1    [Microsoft, Corporation, commonly, referred, M...
2    [Bank, America, American, multinational, banki...
3    [Yahoo, Inc, American, multinational, technolo...
4    [Google, Inc, American, multinational, technol...
Name: Desc, dtype: object

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [10]:
bow_transformer = CountVectorizer(analyzer=text_process).fit(messages['Desc'])

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

139


In [11]:
message1 = messages['Desc'][0]
print(message1)

 Apple Inc. is an American multinational technology company headquartered in Cupertino, California, that designs, develops, and sells consumer electronics, computer software, and online services.


In [12]:
bow1 = bow_transformer.transform([message1])
print(bow1)
print(bow1.shape)

  (0, 7)	1
  (0, 8)	1
  (0, 10)	1
  (0, 15)	1
  (0, 22)	1
  (0, 55)	1
  (0, 57)	1
  (0, 61)	1
  (0, 65)	1
  (0, 66)	1
  (0, 67)	1
  (0, 78)	1
  (0, 96)	1
  (0, 102)	1
  (0, 121)	1
  (0, 123)	1
  (0, 124)	1
  (0, 131)	1
(1, 139)


This means there are 18 unique words in message 1 after removing stop words with each of frequncy 1. The meesage 1 here means the description of company 1

In [13]:
# Compute TF-IDF vectors and cosine similarity

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

documents = [each for each in messages['Desc']]
tfidf = TfidfVectorizer().fit_transform(documents)
# no need to normalize, since Vectorizer will return normalized tf-idf
pairwise_similarity = tfidf * tfidf.T

In [15]:
print pairwise_similarity.A

[[ 1.          0.29451211  0.17996033  0.22461334  0.24547758]
 [ 0.29451211  1.          0.31366557  0.13948072  0.23547846]
 [ 0.17996033  0.31366557  1.          0.1183877   0.19850333]
 [ 0.22461334  0.13948072  0.1183877   1.          0.19836781]
 [ 0.24547758  0.23547846  0.19850333  0.19836781  1.        ]]


## Use similariy method to calcualte the similarity matrix

In [16]:

#### To automate stuff in a method:
def similarity(N_stock):
    messages = pd.read_table('company_descriptions.txt',sep =':'
                           ,names=["Company", "Desc"])
    messages = messages.head(N_stock)
    messages['length'] = messages['Desc'].apply(len)
    messages['Desc'].head(5).apply(text_process)
    documents = [each for each in messages['Desc']]
    tfidf = TfidfVectorizer().fit_transform(documents)
    # no need to normalize, since Vectorizer will return normalized tf-idf
    pairwise_similarity = tfidf * tfidf.T
    return pairwise_similarity.A

In [17]:
similarity(5)

array([[ 1.        ,  0.29451211,  0.17996033,  0.22461334,  0.24547758],
       [ 0.29451211,  1.        ,  0.31366557,  0.13948072,  0.23547846],
       [ 0.17996033,  0.31366557,  1.        ,  0.1183877 ,  0.19850333],
       [ 0.22461334,  0.13948072,  0.1183877 ,  1.        ,  0.19836781],
       [ 0.24547758,  0.23547846,  0.19850333,  0.19836781,  1.        ]])